In [57]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import pandas as pd

In [2]:
engine = create_engine('sqlite:///temp.sqlite', echo = True)

In [4]:
Base = automap_base()

In [5]:
Base.prepare(engine, reflect=True)

2018-01-25 14:47:02,559 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-01-25 14:47:02,562 INFO sqlalchemy.engine.base.Engine ()
2018-01-25 14:47:02,565 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-01-25 14:47:02,567 INFO sqlalchemy.engine.base.Engine ()
2018-01-25 14:47:02,571 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-01-25 14:47:02,572 INFO sqlalchemy.engine.base.Engine ()
2018-01-25 14:47:02,577 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurement")
2018-01-25 14:47:02,580 INFO sqlalchemy.engine.base.Engine ()
2018-01-25 14:47:02,584 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'measurement' AND type = 'table'
2018-01-25 14:47:02,586 INFO sqlalchemy.engine.base.Engine ()
2018-01-25 14:47:02,590 INFO sqlalchemy.

In [6]:
Base.classes.keys()

['measurement', 'station']

In [7]:
Measurement = Base.classes.measurement
Station = Base.classes.station

## Precipitation Analysis

In [58]:
query = "SELECT date, prcp \
FROM measurement \
WHERE date LIKE '2017%' OR date LIKE '2016-12%' OR date LIKE '2016-11%' OR date LIKE '2016-10%' \
OR date LIKE '2016-09%' OR date LIKE '2016-08%' \
LIMIT 10  "
result = engine.execute(query)

2018-01-25 16:24:37,265 INFO sqlalchemy.engine.base.Engine SELECT date, prcp FROM measurement WHERE date LIKE '2017%' OR date LIKE '2016-12%' OR date LIKE '2016-11%' OR date LIKE '2016-10%' OR date LIKE '2016-09%' OR date LIKE '2016-08%' LIMIT 10  
2018-01-25 16:24:37,269 INFO sqlalchemy.engine.base.Engine ()


In [59]:
for r in result:
    print(r.date)

2016-08-01
2016-08-02
2016-08-03
2016-08-04
2016-08-05
2016-08-06
2016-08-07
2016-08-08
2016-08-09
2016-08-10


In [60]:
my_df = pd.read_sql(query, engine)

2018-01-25 16:25:08,840 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("SELECT date, prcp FROM measurement WHERE date LIKE '2017%' OR date LIKE '2016-12%' OR date LIKE '2016-11%' OR date LIKE '2016-10%' OR date LIKE '2016-09%' OR date LIKE '2016-08%' LIMIT 10  ")
2018-01-25 16:25:08,843 INFO sqlalchemy.engine.base.Engine ()
2018-01-25 16:25:08,845 INFO sqlalchemy.engine.base.Engine SELECT date, prcp FROM measurement WHERE date LIKE '2017%' OR date LIKE '2016-12%' OR date LIKE '2016-11%' OR date LIKE '2016-10%' OR date LIKE '2016-09%' OR date LIKE '2016-08%' LIMIT 10  
2018-01-25 16:25:08,847 INFO sqlalchemy.engine.base.Engine ()


In [61]:
my_df.head()

,date,prcp
0,2016-08-01,0.08
1,2016-08-02,0.05
2,2016-08-03,0.00
3,2016-08-04,0.04
4,2016-08-05,0.01
